In [9]:
import pygame as pg
import math
import numpy as np
import random

width, height = 800, 800
debug = False
fps = 60

class Vector:
    def __init__(self, *values):
        self.values = [v for v in values]

    def __add__(self, other):
        result = []
        index = 0
        while index < len(self.values):
            result.append(self.values[index] + other.values[index])
            index += 1
        endVec = Vector(result)
        endVec.values = result
        return endVec

    def __sub__(self, other):
        b = other
        b = b.scale(-1)
        return self+b

    def length(self):
        discriminant = 0
        for i in range(len(self.values)):
            discriminant += (self.values[i])**2
        return discriminant**0.5

    def scale(self, scal):
        new_vals = []
        for i in range(len(self.values)):
            new_vals.append(self.values[i]*scal)
        new_vec = Vector(new_vals)
        new_vec.values = new_vals
        return new_vec


    def __repr__(self):
        result = "["
        for val in self.values:
            result += str(val) + " "
        result += "]"
        return result

class Ball:
    def __init__(self, position, radius=10, velocity=Vector(0, 0), acceleration=Vector(0, 0), color=(255, 255, 255), density=0.1):
        self.position = position
        self.radius = radius
        self.velocity = velocity
        self.acceleration = acceleration
        self.color = color
        self.density = density

    def get_mass(self):
        return self.radius**2*math.pi*self.density

    def dist(self, other):
        discriminant = 0
        for i in range(len(self.position.values)):
            discriminant += (self.position.values[i]-other.position.values[i])**2
        return discriminant**0.5

    def move(self):
        self.velocity = self.velocity + self.acceleration
        self.position = self.position + self.velocity
        #self.velocity = self.velocity.scale(0.999)

    def collide_border(self, startx, starty, endx, endy):
        if(self.position.values[0] - self.radius < startx):
            self.velocity.values[0] *= -1
        if(self.position.values[0] + self.radius > endx):
            self.velocity.values[0] *= -1

        if(self.position.values[1] - self.radius < starty):
            self.velocity.values[1] *= -1
        if(self.position.values[1] + self.radius > endy):
            self.velocity.values[1] *= -1

    def collide_balls(self, balls):
        
        for other_ball in balls:
            if other_ball == self:
                continue
            if self.dist(other_ball) < self.radius+other_ball.radius:
                direction_vector = other_ball.position-self.position
                length = direction_vector.length()
                vel = self.velocity.length()
                other_vel = other_ball.velocity.length()
                other_direction = direction_vector.scale(-1)
                if length > 0:
                    normalized_direction = direction_vector.scale(1/length)
                    other_normal_direction = other_direction.scale(1/length)
                    tangentepunkt = self.position+(normalized_direction.scale(self.radius))
                    normalized_velocity = self.velocity.scale(1/vel)
                    other_normalized_velocity = other_ball.velocity.scale(1/other_vel)
                    if debug:
                        pg.draw.circle(screen, (255, 0, 125), tangentepunkt.values, 6, 2)
                    scalar = normalized_velocity.values[0]*normalized_direction.values[0] + normalized_velocity.values[1]*normalized_direction.values[1]
                    other_scalar = other_normalized_velocity.values[0]*other_normal_direction.values[0] + other_normalized_velocity.values[1]*other_normal_direction.values[1]


                    me_to_other = (self.get_mass()+other_ball.get_mass())/self.get_mass()
                    other_to_me = (self.get_mass()+other_ball.get_mass())/other_ball.get_mass()

                    other_ball.velocity += normalized_direction.scale(scalar*vel*me_to_other/other_to_me)
                    self.velocity -= normalized_direction.scale(scalar*vel*other_to_me/me_to_other)
                    
                    self.velocity += other_normal_direction.scale(other_scalar*other_vel*other_to_me/me_to_other)
                    other_ball.velocity -= other_normal_direction.scale(other_scalar*other_vel*me_to_other/other_to_me)
                    #move balls away from each other
                    self.position += self.velocity
                    other_ball.position += other_ball.velocity
                    #Colliding is not free, that shit costs sum energy $_$
                    self.velocity = self.velocity.scale(0.99)
                    other_ball.velocity = other_ball.velocity.scale(0.99)


    def apply_gravity(self, balls):
        total_gravitational_force = Vector(0, 0)
        for other_ball in balls:
            if other_ball != self:
                d = self.dist(other_ball)
                direction = other_ball.position-self.position
                normal_direction = direction.scale(1/direction.length())
                f = (self.get_mass() + other_ball.get_mass())/(d**2)
                total_gravitational_force+=normal_direction.scale(f/fps)
                if debug:
                    p0 = self.position
                    p1 = self.position + normal_direction.scale(f*fps + self.radius/2)
                    pg.draw.line(screen, (125, 125, 255), (p0.values[0], p0.values[1]), (p1.values[0], p1.values[1]))
        self.velocity += total_gravitational_force
        normal_force = total_gravitational_force.scale(1/total_gravitational_force.length())
        if debug:
            p0 = self.position
            p1 = self.position + normal_force.scale(f*fps*50 + self.radius)
            pg.draw.line(screen, (255, 200, 125), (p0.values[0], p0.values[1]), (p1.values[0], p1.values[1]))

    def draw_screen(self, screen):
        if debug:
            pg.draw.circle(screen, (self.color), (self.position.values[0], self.position.values[1]), self.radius, 2)
            x1 = self.position.values[0]
            y1 = self.position.values[1]
            x2 = self.position.values[0]+self.velocity.values[0]*fps
            y2 = self.position.values[1]+self.velocity.values[1]*fps
            pg.draw.line(screen, (125, 125, 125), (x1, y1), (x2, y2))
        else:
            pg.draw.circle(screen, (self.color), (self.position.values[0], self.position.values[1]), self.radius, 0)


pg.init()
screen = pg.display.set_mode((width, height))
pg.display.set_caption("SUCK MY DICK SEV")
running = True
ballz = []
clock = pg.time.Clock()

#initialisierig oder so
for i in np.arange(0, 1, 1/6):
    vx = math.cos(i*2*math.pi)*0
    vy = math.sin(i*2*math.pi)*0
    radius = 15*random.random()
    density = 1
    ball = Ball(Vector(width/2 + math.cos(2*math.pi*i)*300, height/2 + math.sin(2*math.pi*i)*300),radius=radius,density=density, velocity=Vector(vx, vy))
    ballz.append(ball)

sun = Ball(Vector(width/2, height/2), 20, Vector(0.001, 0), color=(255, 100, 50), density=3)
ballz.append(sun)

while running:
    for event in pg.event.get():
        if event.type == pg.QUIT:
            running = False
        if event.type == pg.KEYDOWN:
            if event.key == pg.K_ESCAPE:
                running = False
            if event.key == pg.K_d:
                debug = not debug
    filler = pg.Surface((width, height), pg.SRCALPHA)
    filler.fill((0, 0, 0, 100))
    screen.blit(filler, (0,0))

    for ball in ballz:
        ball.move()
        ball.collide_border(0, 0, width, height)
        ball.collide_balls(ballz)
        ball.apply_gravity(ballz)
        ball.draw_screen(screen)

    pg.display.flip()
    if debug:
        clock.tick(fps/5)
    else:
        clock.tick(fps)
    
pg.quit()